<a href="https://colab.research.google.com/github/syedimran09/Cat-vs-Dog-Classifier/blob/main/Cat_Dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import zipfile
import random
from pathlib import Path

In [49]:
!pip install kaggle --quiet
!pip install gradio --quiet

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"syedimran09","key":"72a84cd45bfd4c4773ca2318eb1e4cc4"}'}

In [ ]:
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok = True)
shutil.move("kaggle.json", os.path.expanduser("~/.kaggle/kaggle.json"))
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:11<00:00, 143MB/s]
100% 1.06G/1.06G [00:11<00:00, 102MB/s]


In [ ]:
with zipfile.ZipFile("dogs-vs-cats.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset_raw")

In [ ]:
source_cats = Path("dataset_raw/test/cats")
source_dogs = Path("dataset_raw/test/dogs")

In [ ]:
train_cats = Path("dataset/train/cats")
train_dogs = Path("dataset/train/dogs")
val_cats = Path("dataset/validation/cats")
val_dogs = Path("dataset/validation/dogs")

In [ ]:
for path in [train_cats, val_cats, train_dogs, val_dogs]:
  os.makedirs(path, exist_ok = True)

In [ ]:
split_ratio = 0.8

In [ ]:
cat_images = os.listdir(source_cats)
random.shuffle(cat_images)
split_point = int(len(cat_images)*split_ratio)
for img in cat_images[:split_point]:
  shutil.copy(source_cats/img, train_cats/img)
for img in cat_images[split_point:]:
  shutil.copy(source_cats/img, val_cats/img)

In [ ]:
dog_images = os.listdir(source_dogs)
random.shuffle(dog_images)
split_point = int(len(dog_images)*split_ratio)
for img in dog_images[:split_point]:
  shutil.copy(source_dogs/img, train_dogs/img)
for img in dog_images[split_point:]:
  shutil.copy(source_dogs/img, val_dogs/img)

print("Dataset split completed!")

Dataset split completed!


In [59]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size = (150, 150),
    batch_size = 32,
    class_mode = "binary"
)

val_generator = val_datagen.flow_from_directory(
    'dataset/validation',
    target_size = (150, 150),
    batch_size = 32,
    class_mode = "binary"
)

Found 4403 images belonging to 2 classes.
Found 1403 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [64]:
history = model.fit(
    train_generator,
    epochs = 5,
    validation_data = val_generator
)

Epoch 1/5
 14/138 ━━━━━━━━━━━━━━━━━━━━ 4:02 2s/step - accuracy: 0.5597 - loss: 0.6897

KeyboardInterrupt: 

In [66]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label = 'Train Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [51]:
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing import image

In [60]:
def classify_image(img):
    try:
        # Convert image to PIL and resize
        img = img.resize((150, 150))

        # Convert to array
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Prediction
        prediction = model.predict(img_array)[0][0]
        return "Dog 🐶" if prediction > 0.5 else "Cat 🐱"

    except Exception as e:
        print("Error during prediction:", e)
        return f"Error: {str(e)}"


In [61]:
iface = gr.Interface(
    fn=classify_image, inputs=gr.Image(type="pil"), outputs="text",
    title = "Cats vs Dog Classifier",
    description="Upload an image and the AI will tell if it's a cat or Dog"
)
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://46b151766239ff060e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
